### Import dependencies :

In [1]:
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import json
import datetime
from sklearn.preprocessing import StandardScaler , OneHotEncoder
import joblib


2025-06-08 14:26:54.778537: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-08 14:26:55.030002: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-08 14:26:55.223015: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749385615.437327    6825 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749385615.513993    6825 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749385615.873963    6825 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

### Load Data :

In [2]:
data=pd.read_csv('no_outliers.csv')
data

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,g1,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,BOA_RT,alpha
0,0.984234,0.967931,0.849827,0.595673,0.015892,0.402452,0.239730,81.357114,1137.064781,0.370253,0.946187,0.150275,708.015229,3186.068532,0.154730,0.151289,5.695816,3.743539,429.332315,0.133511
1,0.808703,0.502863,0.971936,0.754977,0.212324,0.116173,0.087708,69.669734,2695.217813,0.364243,0.910936,0.347431,708.188358,3186.847609,0.349206,0.347828,2.122562,0.747097,194.210750,0.363279
2,0.266592,0.973465,0.845395,0.580608,1.322036,0.400463,0.232512,17.550390,1081.897319,0.672173,0.896924,0.953452,708.009100,3186.040949,0.953519,0.953467,0.929959,0.610609,648.404343,0.280041
3,0.830782,0.963485,0.971736,0.270474,0.185388,0.039302,0.010630,44.256621,81.149965,0.775990,0.869940,0.716221,707.897906,3185.540575,0.716701,0.716328,1.382759,1.340499,885.138727,0.125099
4,0.600291,0.839312,0.797427,0.652968,0.510340,0.652287,0.425923,24.480144,932.892398,0.777209,0.878946,0.910105,707.992544,3185.966446,0.910238,0.910135,0.990440,0.689159,602.697524,0.320636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362498,0.765135,0.641704,0.963197,0.825442,0.267702,0.214810,0.177313,22.962506,183.428535,0.740243,0.668902,0.920760,707.909270,3185.591714,0.920877,0.920786,1.064013,0.990855,534.140819,0.364438
362499,0.749872,0.800834,0.842750,0.719020,0.287853,0.608888,0.437803,55.826486,452.874619,0.016449,0.818049,0.561701,707.939208,3185.726437,0.562559,0.561892,1.690357,1.419079,387.264753,0.564494
362500,0.538644,0.788402,0.827843,0.727779,0.618701,0.694040,0.505108,6.365350,423.130196,0.050525,0.896188,0.993835,707.935903,3185.711565,0.993844,0.993837,0.959983,0.814335,476.450471,0.591383
362501,0.634018,0.991370,0.965507,0.661306,0.455678,0.103640,0.068538,7.738936,2006.981168,0.832208,0.860886,0.990892,708.111887,3186.503491,0.990905,0.990895,0.807461,0.369966,1015.811664,-0.093105


In [3]:
data['BOA_fraction']=data['BOA_RT']/1000
data

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,BOA_RT,alpha,BOA_fraction
0,0.984234,0.967931,0.849827,0.595673,0.015892,0.402452,0.239730,81.357114,1137.064781,0.370253,...,0.150275,708.015229,3186.068532,0.154730,0.151289,5.695816,3.743539,429.332315,0.133511,0.429332
1,0.808703,0.502863,0.971936,0.754977,0.212324,0.116173,0.087708,69.669734,2695.217813,0.364243,...,0.347431,708.188358,3186.847609,0.349206,0.347828,2.122562,0.747097,194.210750,0.363279,0.194211
2,0.266592,0.973465,0.845395,0.580608,1.322036,0.400463,0.232512,17.550390,1081.897319,0.672173,...,0.953452,708.009100,3186.040949,0.953519,0.953467,0.929959,0.610609,648.404343,0.280041,0.648404
3,0.830782,0.963485,0.971736,0.270474,0.185388,0.039302,0.010630,44.256621,81.149965,0.775990,...,0.716221,707.897906,3185.540575,0.716701,0.716328,1.382759,1.340499,885.138727,0.125099,0.885139
4,0.600291,0.839312,0.797427,0.652968,0.510340,0.652287,0.425923,24.480144,932.892398,0.777209,...,0.910105,707.992544,3185.966446,0.910238,0.910135,0.990440,0.689159,602.697524,0.320636,0.602698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362498,0.765135,0.641704,0.963197,0.825442,0.267702,0.214810,0.177313,22.962506,183.428535,0.740243,...,0.920760,707.909270,3185.591714,0.920877,0.920786,1.064013,0.990855,534.140819,0.364438,0.534141
362499,0.749872,0.800834,0.842750,0.719020,0.287853,0.608888,0.437803,55.826486,452.874619,0.016449,...,0.561701,707.939208,3185.726437,0.562559,0.561892,1.690357,1.419079,387.264753,0.564494,0.387265
362500,0.538644,0.788402,0.827843,0.727779,0.618701,0.694040,0.505108,6.365350,423.130196,0.050525,...,0.993835,707.935903,3185.711565,0.993844,0.993837,0.959983,0.814335,476.450471,0.591383,0.476450
362501,0.634018,0.991370,0.965507,0.661306,0.455678,0.103640,0.068538,7.738936,2006.981168,0.832208,...,0.990892,708.111887,3186.503491,0.990905,0.990895,0.807461,0.369966,1015.811664,-0.093105,1.015812


### Preprocessing :

In [4]:
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import MinMaxScaler

In [5]:
BOA_RT=data['BOA_RT']
data.drop(columns=['BOA_RT'],inplace=True)

In [6]:
alpha = data['alpha']
data.drop(columns=['alpha'],inplace=True)
data

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,g1,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,BOA_fraction
0,0.984234,0.967931,0.849827,0.595673,0.015892,0.402452,0.239730,81.357114,1137.064781,0.370253,0.946187,0.150275,708.015229,3186.068532,0.154730,0.151289,5.695816,3.743539,0.429332
1,0.808703,0.502863,0.971936,0.754977,0.212324,0.116173,0.087708,69.669734,2695.217813,0.364243,0.910936,0.347431,708.188358,3186.847609,0.349206,0.347828,2.122562,0.747097,0.194211
2,0.266592,0.973465,0.845395,0.580608,1.322036,0.400463,0.232512,17.550390,1081.897319,0.672173,0.896924,0.953452,708.009100,3186.040949,0.953519,0.953467,0.929959,0.610609,0.648404
3,0.830782,0.963485,0.971736,0.270474,0.185388,0.039302,0.010630,44.256621,81.149965,0.775990,0.869940,0.716221,707.897906,3185.540575,0.716701,0.716328,1.382759,1.340499,0.885139
4,0.600291,0.839312,0.797427,0.652968,0.510340,0.652287,0.425923,24.480144,932.892398,0.777209,0.878946,0.910105,707.992544,3185.966446,0.910238,0.910135,0.990440,0.689159,0.602698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362498,0.765135,0.641704,0.963197,0.825442,0.267702,0.214810,0.177313,22.962506,183.428535,0.740243,0.668902,0.920760,707.909270,3185.591714,0.920877,0.920786,1.064013,0.990855,0.534141
362499,0.749872,0.800834,0.842750,0.719020,0.287853,0.608888,0.437803,55.826486,452.874619,0.016449,0.818049,0.561701,707.939208,3185.726437,0.562559,0.561892,1.690357,1.419079,0.387265
362500,0.538644,0.788402,0.827843,0.727779,0.618701,0.694040,0.505108,6.365350,423.130196,0.050525,0.896188,0.993835,707.935903,3185.711565,0.993844,0.993837,0.959983,0.814335,0.476450
362501,0.634018,0.991370,0.965507,0.661306,0.455678,0.103640,0.068538,7.738936,2006.981168,0.832208,0.860886,0.990892,708.111887,3186.503491,0.990905,0.990895,0.807461,0.369966,1.015812


In [7]:
scaler=StandardScaler()
scaler_y = MinMaxScaler()
cols_to_scale = data.drop(columns=['BOA_fraction']).columns
X_scaled = scaler.fit_transform(data[cols_to_scale])
y_scaled=scaler_y.fit_transform(data['BOA_fraction'].values.reshape(-1, 1))

In [8]:
X_train,X_test,y_train,y_test=train_test_split(X_scaled,y_scaled,test_size=0.25,random_state=42)

In [30]:
def mae_with_negative_penalty(y_true, y_pred):
    # MAE = moyenne des erreurs absolues
    mae = tf.reduce_mean(tf.abs(y_true - y_pred))
    
    # Pénalité pour les valeurs négatives
    penalty = tf.reduce_mean(tf.maximum(-y_pred, 0.0)) * 5  # ajuste le facteur selon tes besoins

    return mae + penalty


In [31]:
with open("best_params.json", "r") as f:
    params = json.load(f)

# Build the model
model = Sequential()

# Layer 1
model.add(Dense(params["units1"], activation=params["activation"], input_shape=(X_train.shape[1],)))
model.add(Dropout(params["dropout1"]))

# Layer 2
model.add(Dense(params["units2"], activation=params["activation"]))
model.add(Dropout(params["dropout2"]))

# Layer 3 (if applicable)
if params["n_layers"] >= 3:
    model.add(Dense(params["units3"], activation=params["activation"]))
    model.add(Dropout(params["dropout3"]))

# Layer 4 (if applicable)
if params["n_layers"] == 4:
    model.add(Dense(params["units4"], activation=params["activation"]))
    model.add(Dropout(params["dropout4"]))

# Output layer
model.add(Dense(1))
optimizer = tf.keras.optimizers.Adam(learning_rate=params["lr"])
model.compile(optimizer=optimizer, loss=mae_with_negative_penalty, metrics=['mae'])

# Affichage du résumé du modèle
model.summary()




Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 232)            │         4,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 232)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 249)            │        58,017 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 249)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 174)            │        43,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 174)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           175 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 106,100 (414.45 KB)

 Trainable params: 106,100 (414.45 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
# 1. Early stopping
early_stop = EarlyStopping(
    monitor='val_mae',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

# 2. Reduce learning rate on plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_mae',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

# 3. Model checkpoint (save best model)
checkpoint_cb = ModelCheckpoint(
    filepath='deep_model.keras',
    monitor='val_mae',
    save_best_only=True,
    verbose=1
)

# 4. CSV logger (log training history)
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
csv_logger = CSVLogger(f'training_log_{timestamp}.csv')

# Train phase :

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=params["batch_size"],
    callbacks=[early_stop, reduce_lr, checkpoint_cb, csv_logger],
    verbose=1
)


Epoch 1/200


3399/3399 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1187 - mae: 0.1045
Epoch 1: val_mae improved from inf to 0.02471, saving model to deep_model.keras
3399/3399 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - loss: 0.1187 - mae: 0.1044 - val_loss: 0.0247 - val_mae: 0.0247 - learning_rate: 3.4066e-04
Epoch 2/200
3398/3399 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0331 - mae: 0.0324
Epoch 2: val_mae improved from 0.02471 to 0.01450, saving model to deep_model.keras
3399/3399 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - loss: 0.0331 - mae: 0.0324 - val_loss: 0.0146 - val_mae: 0.0145 - learning_rate: 3.4066e-04
Epoch 3/200
3394/3399 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0222 - mae: 0.0220
Epoch 3: val_mae improved from 0.01450 to 0.01069, saving model to deep_model.keras
3399/3399 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - loss: 0.0222 - mae: 0.0220 - val_loss: 0.0107 - val_mae: 0.0107 - learning_rate: 3.4066e-04
Epoch 4/200
3390/3399 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0176 - mae: 0.0174
Epoch 4: val_m

In [33]:
y_pred=model.predict(X_test)

2833/2833 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step


In [13]:
def inverse_scaling(X,scaler,y,scaler_y):
    '''cols_to_scale_inv = X.drop(columns=['mprime_a_bin_0', 'mprime_a_bin_1',
       'mprime_a_bin_2', 'mprime_a_bin_3', 'mprime_a_bin_4', 'mprime_g_bin_0',
       'mprime_g_bin_1', 'mprime_g_bin_2', 'mprime_g_bin_3', 'mprime_g_bin_4'
       ]).columns
    cols_to_encode_inv = ['mprime_a_bin_0', 'mprime_a_bin_1',
       'mprime_a_bin_2', 'mprime_a_bin_3', 'mprime_a_bin_4', 'mprime_g_bin_0',
       'mprime_g_bin_1', 'mprime_g_bin_2', 'mprime_g_bin_3', 'mprime_g_bin_4',
       ]'''
    #X_scaled_part = X_test[cols_to_scale_inv].values
    #X_encoded_part = X_test[cols_to_encode_inv].values
    # 2. Inverser la standardisation
    X_original_scaled = scaler.inverse_transform(X)

    # 3. Inverser l'encodage
    #X_original_categoricals = encoder.inverse_transform(X_encoded_part)
    X_real= pd.DataFrame(
        X_original_scaled,
        columns=['Tg_scat', 'Tg_abs', 'Ta_abs', 'SSA', 'GOD', 'AOD', 'AODS', 'SZA', 'Z',
        'R_scence', 'g1', 'Cos(SZA)', 'mu_g', 'mu_a', 'muprime_g', 'muprime_a',
        'mprime_g', 'mprime_a'],
        index=X_scaled.index if isinstance(X_scaled, pd.DataFrame) else None
    )
    '''df_categoricals = pd.DataFrame(
        X_original_categoricals,
        columns=['mprime_a_bin', 'mprime_g_bin'],
        index=X_test.index
    )'''
    y_real=scaler_y.inverse_transform(y)
    return X_real , y_real

In [34]:
X_test_real,y_test_real=inverse_scaling(X_test,scaler,y_test,scaler_y)

In [1]:
def k_gaz_TROPICAL():
    x = np.linspace(0, 2.0, 1000)  

    weights = np.piecewise(
        x,
        [x < 1.37, 
         (x >= 1.38) & (x < 1.5), 
         (x >= 1.5) & (x <= 1.75),
         x > 1.75],
        [0,  # Avant 1.38, poids nul
         1,  # Plateau constant
         lambda x: (1.75 - x) / (1.75 - 1.5),  # Descente linéaire
         0]  # Après 1.75, poids nul
    )

    weights /= weights.sum()

    return np.random.choice(x, size=1, p=weights)[0]

In [4]:
def K_aerosol():
    x = np.linspace(0, 7, 1000)
    
    # Accentuer la montée vers 1 et la descente après 1
    weights = np.piecewise(
        x,
        [x <= 1, x > 1],
        [
            lambda x: (x / 1) ** 2,        # Monte plus vite vers 1
            lambda x: ((7 - x) / (7 - 1)) ** 2  # Descend plus vite après 1
        ]
    )

    weights = np.maximum(weights, 0)  # sécurité
    weights /= weights.sum()

    return np.random.choice(x, size=1, p=weights)[0]

In [5]:
import numpy as np
for i in range(1000):
    print(K_aerosol())

1.107107107107107
2.5855855855855854
1.8078078078078077
0.8758758758758759
0.924924924924925
2.6486486486486487
2.5435435435435436
1.8638638638638638
1.4854854854854855
1.6956956956956957
4.414414414414415
1.2822822822822824
1.4504504504504505
1.3943943943943944
2.3473473473473474
0.8758758758758759
3.965965965965966
1.7797797797797799
1.4504504504504505
2.774774774774775
4.7857857857857855
3.4754754754754753
0.5045045045045045
3.2862862862862863
2.4804804804804803
2.011011011011011
0.5885885885885885
2.067067067067067
1.5905905905905906
1.5135135135135136
0.924924924924925
4.4774774774774775
1.1561561561561562
2.2982982982982985
0.4904904904904905
2.991991991991992
1.212212212212212
5.675675675675675
4.344344344344345
1.093093093093093
4.078078078078078
2.2072072072072073
2.3753753753753752
3.5105105105105103
1.2472472472472473
4.155155155155155
5.017017017017017
0.8338338338338338
0.6446446446446447
0.5325325325325325
2.158158158158158
1.5345345345345345
4.183183183183183
1.212212212

In [35]:
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,g1,Cos(SZA),mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a
0,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,0.849374,0.951706,708.317583,3187.429121,0.951776,0.951721,0.684362,0.152645
1,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,0.814209,0.470531,708.012549,3186.056472,0.471695,0.470791,1.873413,1.217586
2,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,0.915442,0.240763,707.890641,3185.507886,0.243492,0.241375,4.099723,4.110381
3,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,0.852120,0.453122,708.341159,3187.535216,0.454355,0.453397,1.400187,0.288163
4,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,0.869006,0.999158,707.958273,3185.812230,0.999159,0.999158,0.933753,0.732430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90621,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,0.959487,0.962254,708.064025,3186.288112,0.962308,0.962266,0.872219,0.472533
90622,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,0.812618,0.025776,708.122465,3186.551091,0.042399,0.030856,18.672395,11.328534
90623,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,0.789790,0.909007,707.894025,3185.523112,0.909141,0.909037,1.094304,1.074933
90624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,0.976101,0.859409,708.028352,3186.127582,0.859623,0.859456,1.011868,0.621184


In [36]:
y_test_real

array([[0.91444475],
       [0.5945755 ],
       [0.28459194],
       ...,
       [0.73562703],
       [0.92437937],
       [0.73044082]])

In [37]:
y_pred_real=scaler_y.inverse_transform(y_pred)
y_pred_real

array([[0.915453  ],
       [0.59725976],
       [0.28065583],
       ...,
       [0.73353857],
       [0.92147213],
       [0.73259103]], dtype=float32)

In [38]:
y_test_real

array([[0.91444475],
       [0.5945755 ],
       [0.28459194],
       ...,
       [0.73562703],
       [0.92437937],
       [0.73044082]])

In [39]:
X_test_real['BOA_fraction']=y_test_real
X_test_real['BOA_fraction_pred']=y_pred_real

In [40]:
X_test_real['BOA_RT']=X_test_real['BOA_fraction']*1000
X_test_real['BOA_pred']=X_test_real['BOA_fraction_pred']*1000
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,BOA_fraction,BOA_fraction_pred,BOA_RT,BOA_pred
0,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,708.317583,3187.429121,0.951776,0.951721,0.684362,0.152645,0.914445,0.915453,914.444755,915.453003
1,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,708.012549,3186.056472,0.471695,0.470791,1.873413,1.217586,0.594575,0.597260,594.575497,597.259766
2,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,707.890641,3185.507886,0.243492,0.241375,4.099723,4.110381,0.284592,0.280656,284.591943,280.655823
3,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,708.341159,3187.535216,0.454355,0.453397,1.400187,0.288163,0.373719,0.373551,373.718733,373.550720
4,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,707.958273,3185.812230,0.999159,0.999158,0.933753,0.732430,0.832138,0.833302,832.137702,833.301880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90621,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,708.064025,3186.288112,0.962308,0.962266,0.872219,0.472533,0.801588,0.801431,801.588284,801.431396
90622,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,708.122465,3186.551091,0.042399,0.030856,18.672395,11.328534,0.032089,0.045344,32.089238,45.344391
90623,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,707.894025,3185.523112,0.909141,0.909037,1.094304,1.074933,0.735627,0.733539,735.627029,733.538574
90624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,708.028352,3186.127582,0.859623,0.859456,1.011868,0.621184,0.924379,0.921472,924.379368,921.472107


In [41]:
X_test_real[X_test_real['BOA_pred']<0]

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,BOA_fraction,BOA_fraction_pred,BOA_RT,BOA_pred
6066,0.948453,0.081226,0.979127,0.989486,0.052923,2.006293,1.985198,85.79666,339.707196,0.475261,...,707.926634,3185.669854,0.081866,0.075367,11.762631,11.195724,0.000157,-0.002975,0.15682,-2.974701


In [23]:
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mu_g,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,BOA_fraction,BOA_fraction_pred,BOA_RT,BOA_pred
0,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,708.317583,3187.429121,0.951776,0.951721,0.684362,0.152645,0.914445,0.917256,914.444755,917.255554
1,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,708.012549,3186.056472,0.471695,0.470791,1.873413,1.217586,0.594575,0.595136,594.575497,595.135681
2,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,707.890641,3185.507886,0.243492,0.241375,4.099723,4.110381,0.284592,0.280470,284.591943,280.470062
3,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,708.341159,3187.535216,0.454355,0.453397,1.400187,0.288163,0.373719,0.373296,373.718733,373.295898
4,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,707.958273,3185.812230,0.999159,0.999158,0.933753,0.732430,0.832138,0.832918,832.137702,832.917542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90621,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,708.064025,3186.288112,0.962308,0.962266,0.872219,0.472533,0.801588,0.800043,801.588284,800.042725
90622,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,708.122465,3186.551091,0.042399,0.030856,18.672395,11.328534,0.032089,0.048756,32.089238,48.756180
90623,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,707.894025,3185.523112,0.909141,0.909037,1.094304,1.074933,0.735627,0.733867,735.627029,733.867493
90624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,708.028352,3186.127582,0.859623,0.859456,1.011868,0.621184,0.924379,0.919594,924.379368,919.593994


In [56]:
import math
def calculate_BOA(thetha,z,Tg_abs,Tg_scat,optical_depth,albedo,alpha):   
    def muprime(z,h,µ):
        RAYON_TERRESTRE=6371
        eta = (RAYON_TERRESTRE*1000 + z) / h
        root = (eta*µ)**2  + 2 * eta + 1
        sum = (root)**0.5 - eta * µ
        if sum > 0 :
            return 1/sum
        return 1 
    Ha=2000
    Hg=9000
    angle_rad = math.radians(thetha)
    µ=math.cos(angle_rad)  
    Y_a=muprime(z,Ha,µ)
    Y_g=muprime(z,Hg,µ)
    Ma=math.exp(-z/Ha)/Y_a
    Mg=math.exp(-z/Hg)/Y_g
    delta_a_scat=optical_depth*albedo
    delta_g_scat=-math.log(Tg_scat)
    Ta_abs=math.exp(-optical_depth*(1-albedo))
    numerator= 1000*(Tg_abs**Mg)*(Ta_abs**Ma)
    denominator=1+alpha*delta_g_scat*Mg+(alpha*(1/3)*delta_a_scat)*Ma
    BOA_ia= numerator / denominator
    return BOA_ia

In [24]:
def calculate_alpha(BOA_fraction,Ta_abs,Tg_abs,mprime_g,mprime_a,GOD,AODS):   
    numerator= (1/BOA_fraction)*(Tg_abs**(mprime_g))*(Ta_abs**(mprime_a))-1
    denominator=(GOD * mprime_g) + (AODS * mprime_a) / 3
    alpha= numerator / denominator
    return alpha

In [25]:
X_test_real['alpha_pred']=X_test_real.apply(
    lambda row: calculate_alpha(
        row['BOA_fraction_pred'],
        row['Ta_abs'],
        row['Tg_abs'],
        row['mprime_g'],
        row['mprime_a'],
        row['GOD'],
        row['AODS'],
    ), axis=1
)
X_test_real['alpha']=X_test_real.apply(
    lambda row: calculate_alpha(
        row['BOA_fraction'],
        row['Ta_abs'],
        row['Tg_abs'],
        row['mprime_g'],
        row['mprime_a'],
        row['GOD'],
        row['AODS'],
    ), axis=1
)
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,muprime_g,muprime_a,mprime_g,mprime_a,BOA_fraction,BOA_fraction_pred,BOA_RT,BOA_pred,alpha_pred,alpha
0,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,0.951776,0.951721,0.684362,0.152645,0.914445,0.917256,914.444755,917.255554,-0.417386,-0.224256
1,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,0.471695,0.470791,1.873413,1.217586,0.594575,0.595136,594.575497,595.135681,0.375515,0.376680
2,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,0.243492,0.241375,4.099723,4.110381,0.284592,0.280470,284.591943,280.470062,0.257075,0.238902
3,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,0.454355,0.453397,1.400187,0.288163,0.373719,0.373296,373.718733,373.295898,0.475234,0.471005
4,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,0.999159,0.999158,0.933753,0.732430,0.832138,0.832918,832.137702,832.917542,0.125017,0.128247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90621,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,0.962308,0.962266,0.872219,0.472533,0.801588,0.800043,801.588284,800.042725,0.215932,0.194530
90622,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,0.042399,0.030856,18.672395,11.328534,0.032089,0.048756,32.089238,48.756180,-0.142424,-0.142424
90623,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,0.909141,0.909037,1.094304,1.074933,0.735627,0.733867,735.627029,733.867493,0.168416,0.149952
90624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,0.859623,0.859456,1.011868,0.621184,0.924379,0.919594,924.379368,919.593994,0.166861,0.143431


In [58]:
X_test_real['BOA_pred'] = X_test_real.apply(lambda row: calculate_BOA(row['SZA'], row['Z'],row['Tg_abs'],row['Tg_scat'],row['AOD'],row['SSA'],row['alpha_pred']), axis=1)
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,muprime_a,mprime_g,mprime_a,BOA_fraction,BOA_fraction_pred,alpha_pred,alpha,BOA_pred,BOA_RT,BOA_RT_test
0,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,0.951721,0.684362,0.152645,0.914445,0.915612,-0.304591,-0.224256,915.611863,914.444755,914.444755
1,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,0.470791,1.873413,1.217586,0.594575,0.594615,0.376599,0.376680,594.614685,594.575497,594.575497
2,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,0.241375,4.099723,4.110381,0.284592,0.280167,0.258431,0.238902,280.167222,284.591943,284.591943
3,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,0.453397,1.400187,0.288163,0.373719,0.372900,0.479201,0.471005,372.900099,373.718733,373.718733
4,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,0.999158,0.933753,0.732430,0.832138,0.834354,0.119085,0.128247,834.353626,832.137702,832.137702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90621,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,0.962266,0.872219,0.472533,0.801588,0.800986,0.202855,0.194530,800.986350,801.588284,801.588284
90622,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,0.030856,18.672395,11.328534,0.032089,0.036499,-0.142424,-0.142424,36.498807,32.089238,32.089238
90623,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,0.909037,1.094304,1.074933,0.735627,0.734697,0.159697,0.149952,734.697282,735.627029,735.627029
90624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,0.859456,1.011868,0.621184,0.924379,0.922319,0.153491,0.143431,922.318637,924.379368,924.379368


In [49]:
def calculate_BOA_IA(Tg_abs,Mg,Ta_abs,Ma,alpha,delta_g_scat,AODS):
    numerator= 1000*(Tg_abs**Mg)*(Ta_abs**Ma)
    denominator=1+alpha*delta_g_scat*Mg+(alpha*(1/3)*AODS)*Ma
    
    BOA_ia= numerator / denominator
    return BOA_ia


In [50]:
X_test_real['BOA_pred'] = X_test_real.apply(lambda row: calculate_BOA_IA(row['Tg_abs'], row['mprime_g'],row['Ta_abs'],row['mprime_a'],row['alpha_pred'],row['GOD'],row['AODS']), axis=1)
X_test_real['BOA_RT']=1000*X_test_real['BOA_fraction']

In [53]:
X_test_real['BOA_RT_test'] = X_test_real.apply(lambda row: calculate_BOA_IA(row['Tg_abs'], row['mprime_g'],row['Ta_abs'],row['mprime_a'],row['alpha'],row['GOD'],row['AODS']), axis=1)


In [54]:
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,muprime_a,mprime_g,mprime_a,BOA_fraction,BOA_fraction_pred,alpha_pred,alpha,BOA_pred,BOA_RT,BOA_RT_test
0,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,0.951721,0.684362,0.152645,0.914445,0.915612,-0.304591,-0.224256,915.611863,914.444755,914.444755
1,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,0.470791,1.873413,1.217586,0.594575,0.594615,0.376599,0.376680,594.614685,594.575497,594.575497
2,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,0.241375,4.099723,4.110381,0.284592,0.280167,0.258431,0.238902,280.167222,284.591943,284.591943
3,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,0.453397,1.400187,0.288163,0.373719,0.372900,0.479201,0.471005,372.900099,373.718733,373.718733
4,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,0.999158,0.933753,0.732430,0.832138,0.834354,0.119085,0.128247,834.353626,832.137702,832.137702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90621,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,0.962266,0.872219,0.472533,0.801588,0.800986,0.202855,0.194530,800.986350,801.588284,801.588284
90622,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,0.030856,18.672395,11.328534,0.032089,0.036499,-0.142424,-0.142424,36.498807,32.089238,32.089238
90623,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,0.909037,1.094304,1.074933,0.735627,0.734697,0.159697,0.149952,734.697282,735.627029,735.627029
90624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,0.859456,1.011868,0.621184,0.924379,0.922319,0.153491,0.143431,922.318637,924.379368,924.379368


### Alpha Erreur :

In [26]:
from sklearn.metrics import mean_absolute_error
mae=mean_absolute_error(X_test_real['alpha'],X_test_real['alpha_pred'])
mae

0.056358177696628375

### E_BOA Erreur :

In [27]:
from sklearn.metrics import mean_absolute_error
mae=mean_absolute_error(X_test_real['BOA_RT'],X_test_real['BOA_pred'])
mae

3.3361722759313124

In [28]:
X_test_real['Erreur']=np.abs(X_test_real['BOA_RT']-X_test_real['BOA_pred'])
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,muprime_a,mprime_g,mprime_a,BOA_fraction,BOA_fraction_pred,BOA_RT,BOA_pred,alpha_pred,alpha,Erreur
0,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,0.951721,0.684362,0.152645,0.914445,0.917256,914.444755,917.255554,-0.417386,-0.224256,2.810799
1,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,0.470791,1.873413,1.217586,0.594575,0.595136,594.575497,595.135681,0.375515,0.376680,0.560184
2,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,0.241375,4.099723,4.110381,0.284592,0.280470,284.591943,280.470062,0.257075,0.238902,4.121881
3,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,0.453397,1.400187,0.288163,0.373719,0.373296,373.718733,373.295898,0.475234,0.471005,0.422835
4,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,0.999158,0.933753,0.732430,0.832138,0.832918,832.137702,832.917542,0.125017,0.128247,0.779840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90621,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,0.962266,0.872219,0.472533,0.801588,0.800043,801.588284,800.042725,0.215932,0.194530,1.545559
90622,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,0.030856,18.672395,11.328534,0.032089,0.048756,32.089238,48.756180,-0.142424,-0.142424,16.666942
90623,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,0.909037,1.094304,1.074933,0.735627,0.733867,735.627029,733.867493,0.168416,0.149952,1.759536
90624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,0.859456,1.011868,0.621184,0.924379,0.919594,924.379368,919.593994,0.166861,0.143431,4.785374


In [29]:
X_test_real[X_test_real['Erreur']>=20]

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,muprime_a,mprime_g,mprime_a,BOA_fraction,BOA_fraction_pred,BOA_RT,BOA_pred,alpha_pred,alpha,Erreur
158,0.339585,0.069493,0.814930,0.215956,1.080031,0.261022,0.056369,38.372890,3885.035877,0.223047,...,0.784064,0.827994,0.182820,0.060016,0.094407,60.015525,94.407310,0.135579,0.851628,34.391785
224,0.990893,0.972872,0.993285,0.677149,0.009149,0.020870,0.014132,14.520411,6968.795702,0.306753,...,0.968069,0.476211,0.031684,0.985817,1.008013,985.817193,1008.012878,-4.675118,0.216218,22.195685
255,0.788627,0.115739,0.969286,0.349916,0.237462,0.047987,0.016792,13.828972,113.301969,0.921797,...,0.971023,1.016925,0.973122,0.092290,0.113015,92.289962,113.015335,-0.170595,0.700542,20.725373
328,0.954422,0.955509,0.987617,0.052284,0.046650,0.013148,0.000687,86.528727,449.272884,0.508727,...,0.063028,13.490962,12.673768,0.435286,0.408940,435.286133,408.939880,0.205724,0.097541,26.346253
523,0.983471,0.875427,0.982249,0.813108,0.016667,0.095834,0.077924,78.652311,14.654955,0.076201,...,0.197526,4.988127,5.025667,0.434292,0.411813,434.292294,411.812744,0.668605,0.391756,22.479550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90363,0.981290,0.999815,0.979137,0.539137,0.018887,0.045748,0.024664,89.208355,662.927875,0.313969,...,0.021212,27.043527,33.842834,0.486492,0.426726,486.491882,426.726379,0.180399,0.002536,59.765503
90436,0.840182,0.964685,0.912374,0.929614,0.174137,1.302896,1.211190,35.339254,421.807833,0.364517,...,0.815806,1.169336,0.992702,0.625478,0.660142,625.477792,660.142273,0.539491,0.661084,34.664481
90471,0.934574,0.990226,0.962033,0.207586,0.067665,0.048846,0.010140,87.813810,1305.229228,0.687272,...,0.041887,16.713342,12.430686,0.489099,0.464213,489.099111,464.212738,0.110732,0.061717,24.886373
90474,0.966291,0.775944,0.951105,0.894507,0.034291,0.475204,0.425074,84.213252,1124.497811,0.947106,...,0.102344,8.222727,5.568754,0.112914,0.133656,112.913699,133.655716,-0.277423,-0.156866,20.742017


In [60]:
import numpy as np

# Vérifie les infinis
print("BOA_RT contient Inf :", np.isinf(X_test_real['BOA_RT']).any())
print("BOA_pred contient Inf :", np.isinf(X_test_real['BOA_pred']).any())

# Vérifie les NaNs
print("BOA_RT contient NaN :", X_test_real['BOA_RT'].isna().any())
print("BOA_pred contient NaN :", X_test_real['BOA_pred'].isna().any())


BOA_RT contient Inf : False
BOA_pred contient Inf : True
BOA_RT contient NaN : False
BOA_pred contient NaN : False


In [61]:
def calculate_BOA_IA(Tg_abs,Mg,Ta_abs,Ma,alpha,delta_g_scat,AODS):
    numerator= 1000*(Tg_abs**Mg)*(Ta_abs**Ma)
    denominator=1+alpha*delta_g_scat*Mg+(alpha*(1/3)*AODS)*Ma
    
    BOA_ia= numerator / denominator
    return BOA_ia

In [52]:
infs = X_test_real[np.isinf(X_test_real['BOA_pred'])]
print("Lignes avec Inf dans BOA_pred :")
infs


Lignes avec Inf dans BOA_pred :


,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,muprime_g,muprime_a,mprime_g,mprime_a,BOA_fraction,BOA_fraction_pred,alpha_pred,alpha,BOA_pred,BOA_RT
2379,0.877229,0.152220,0.831589,0.852392,0.130988,1.249368,1.064951,89.985541,2877.306578,0.876668,...,0.026688,0.012651,27.217166,18.752627,0.002612,-0.013425,-0.097828,-0.097828,inf,2.612085
8289,0.975981,0.272728,0.861939,0.748362,0.024312,0.590417,0.441845,89.423959,513.089170,0.777840,...,0.032060,0.018524,29.462709,41.769395,0.000809,-0.020888,-0.145599,-0.145599,inf,0.808511
17860,0.956120,0.467668,0.636342,0.447400,0.044872,0.817985,0.365966,89.367327,222.750242,0.010789,...,0.032651,0.019210,29.878344,46.570483,0.003245,-0.012042,-0.142414,-0.142414,inf,3.245207
28819,0.757926,0.091870,0.873886,0.772973,0.277170,0.593786,0.458981,89.614225,45.818359,0.384029,...,0.030144,0.016338,33.006107,59.821935,0.000754,-0.014707,-0.054643,-0.054643,inf,0.754105
29336,0.791410,0.491256,0.592520,0.434791,0.233939,0.925977,0.402607,89.669515,625.046765,0.175599,...,0.029604,0.015738,31.512932,46.486176,0.015496,0.023010,-0.073472,-0.073472,inf,15.495546
32822,0.862867,0.433229,0.858311,0.249329,0.147494,0.203537,0.050748,89.915112,62.657872,0.714013,...,0.027318,0.013290,36.352136,72.924195,0.011157,0.011503,-0.151623,-0.151623,inf,11.156664
34028,0.913785,0.233169,0.953676,0.632550,0.090160,0.129082,0.081651,89.867727,177.226526,0.019382,...,0.027746,0.013734,35.339028,66.635415,0.002142,-0.018445,-0.200009,-0.200009,inf,2.141626
35288,0.365234,0.202715,0.751287,0.651047,1.007217,0.819501,0.533533,89.757370,484.501151,0.540442,...,0.028766,0.014822,32.940904,52.953818,0.005638,-0.001282,-0.023476,-0.023476,inf,5.638409
50363,0.613289,0.312638,0.627556,0.499441,0.488918,0.930805,0.464883,88.766153,19.530294,0.299897,...,0.039422,0.027282,25.311487,36.297997,0.006618,0.000214,-0.055555,-0.055555,inf,6.617930
61655,0.969413,0.103506,0.966317,0.919750,0.031065,0.426964,0.392701,89.929943,241.982808,0.792710,...,0.027185,0.013153,35.809323,67.362341,0.000163,-0.034433,-0.100703,-0.100703,inf,0.163120


In [41]:
X_test['alpha_pred']=X_test.apply(
    lambda row: calculate_alpha(
        row['BOA_fraction_pred'],
        row['Ta_abs'],
        row['Tg_abs'],
        row['mprime_g'],
        row['mprime_a'],
        row['GOD'],
        row['AODS'],
    ), axis=1
)
X_test

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,BOA_RT,alpha,BOA_fraction_pred,BOA_fraction,alpha_pred
159680,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,3187.429121,0.951776,0.951721,0.684362,0.152645,914.444755,-0.224256,0.896171,0.914445,1.060831
347342,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,3186.056472,0.471695,0.470791,1.873413,1.217586,594.575497,0.376680,0.601568,0.594575,0.362287
307069,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,3185.507886,0.243492,0.241375,4.099723,4.110381,284.591943,0.238902,0.290517,0.284592,0.213682
126659,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,3187.535216,0.454355,0.453397,1.400187,0.288163,373.718733,0.471005,0.380374,0.373719,0.405693
356896,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,3185.812230,0.999159,0.999158,0.933753,0.732430,832.137702,0.128247,0.829379,0.832138,0.139722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326002,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,3186.288112,0.962308,0.962266,0.872219,0.472533,801.588284,0.194530,0.798049,0.801588,0.243669
190999,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,3186.551091,0.042399,0.030856,18.672395,11.328534,32.089238,-0.142424,0.092493,0.032089,-0.142425
27703,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,3185.523112,0.909141,0.909037,1.094304,1.074933,735.627029,0.149952,0.732899,0.735627,0.178618
76624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,3186.127582,0.859623,0.859456,1.011868,0.621184,924.379368,0.143431,0.906780,0.924379,0.230817


In [42]:
X_test['BOA_RT_pred'] = X_test['BOA_fraction_pred'] * 1000
X_test

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,muprime_g,muprime_a,mprime_g,mprime_a,BOA_RT,alpha,BOA_fraction_pred,BOA_fraction,alpha_pred,BOA_RT_pred
159680,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,0.951776,0.951721,0.684362,0.152645,914.444755,-0.224256,0.896171,0.914445,1.060831,896.171387
347342,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,0.471695,0.470791,1.873413,1.217586,594.575497,0.376680,0.601568,0.594575,0.362287,601.568298
307069,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,0.243492,0.241375,4.099723,4.110381,284.591943,0.238902,0.290517,0.284592,0.213682,290.516998
126659,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,0.454355,0.453397,1.400187,0.288163,373.718733,0.471005,0.380374,0.373719,0.405693,380.373718
356896,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,0.999159,0.999158,0.933753,0.732430,832.137702,0.128247,0.829379,0.832138,0.139722,829.379211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326002,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,0.962308,0.962266,0.872219,0.472533,801.588284,0.194530,0.798049,0.801588,0.243669,798.048645
190999,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,0.042399,0.030856,18.672395,11.328534,32.089238,-0.142424,0.092493,0.032089,-0.142425,92.492775
27703,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,0.909141,0.909037,1.094304,1.074933,735.627029,0.149952,0.732899,0.735627,0.178618,732.898804
76624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,0.859623,0.859456,1.011868,0.621184,924.379368,0.143431,0.906780,0.924379,0.230817,906.780029


In [43]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(X_test['BOA_RT'], X_test['BOA_RT_pred'])
mae

12.739169428433469

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class TransmissionLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        alpha, m_g, m_a, dtau_gas_scat, dtau_aero_scat, T_gas_abs, T_aero_abs = inputs

        # Dénominateur (1 + α⋅Δτ_gas⋅m_g + α/3⋅Δτ_aero⋅m_a)
        denom = 1 + alpha * dtau_gas_scat * m_g + (alpha / 3.0) * dtau_aero_scat * m_a

        # Absorptions
        T_abs = tf.pow(T_gas_abs, m_g) * tf.pow(T_aero_abs, m_a)

        # E_BOA / E_TOA
        E_BOA_div_TOA = (T_abs/ denom) 
        return E_BOA_div_TOA

In [17]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout

# Entrée principale : 18 descripteurs
features_input = Input(shape=(18,), name='features')

# Autres entrées nécessaires à la formule analytique
m_g_input = Input(shape=(1,), name='mprime_g')
m_a_input = Input(shape=(1,), name='mprime_a')
dtau_gas_scat_input = Input(shape=(1,), name='GOD')
dtau_aero_scat_input = Input(shape=(1,), name='AODS')
T_gas_abs_input = Input(shape=(1,), name='Tg_abs')
T_aero_abs_input = Input(shape=(1,), name='Ta_abs')

In [18]:
x = Dense(232, activation='elu')(features_input)
x = Dropout(0.111)(x)
x = Dense(249, activation='elu')(x)
x = Dropout(0.138)(x)
x = Dense(174, activation='elu')(x)
x = Dropout(0.136)(x)

# Prédiction de alpha
alpha_output = Dense(1, name='alpha')(x)


In [19]:
E_BOA_div_TOA = TransmissionLayer()([
    alpha_output,
    m_g_input,
    m_a_input,
    dtau_gas_scat_input,
    dtau_aero_scat_input,
    T_gas_abs_input,
    T_aero_abs_input
])
model = Model(
    inputs=[features_input, m_g_input, m_a_input, dtau_gas_scat_input, dtau_aero_scat_input, T_gas_abs_input, T_aero_abs_input],
    outputs=E_BOA_div_TOA
)

In [50]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00034),
    loss='mae',  # ou 'mae' selon ce que tu veux
    metrics=['mae']
)

In [10]:
# 1. Early stopping
early_stop = EarlyStopping(
    monitor='val_mae',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

# 2. Reduce learning rate on plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_mae',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

# 3. Model checkpoint (save best model)
checkpoint_cb = ModelCheckpoint(
    filepath='deep_model.keras',
    monitor='val_mae',
    save_best_only=True,
    verbose=1
)

# 4. CSV logger (log training history)
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
csv_logger = CSVLogger(f'training_log_{timestamp}.csv')

# Train phase :

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=params["batch_size"],
    callbacks=[early_stop, reduce_lr, checkpoint_cb, csv_logger],
    verbose=1
)


Epoch 1/200
3398/3399 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1676 - mae: 0.1676
Epoch 1: val_mae improved from inf to 0.04998, saving model to deep_model.keras
3399/3399 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - loss: 0.1675 - mae: 0.1675 - val_loss: 0.0500 - val_mae: 0.0500 - learning_rate: 3.4066e-04
Epoch 2/200
3395/3399 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0778 - mae: 0.0778
Epoch 2: val_mae improved from 0.04998 to 0.04223, saving model to deep_model.keras
3399/3399 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - loss: 0.0778 - mae: 0.0778 - val_loss: 0.0422 - val_mae: 0.0422 - learning_rate: 3.4066e-04
Epoch 3/200
3396/3399 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0638 - mae: 0.0638
Epoch 3: val_mae improved from 0.04223 to 0.02865, saving model to deep_model.keras
3399/3399 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - loss: 0.0638 - mae: 0.0638 - val_loss: 0.0287 - val_mae: 0.0287 - learning_rate: 3.4066e-04
Epoch 4/200
3395/3399 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0596 - mae: 0.0596
Ep

KeyboardInterrupt: 

In [ ]:
X_features = data.iloc[:, :18].values

# autres colonnes utilisées dans la couche analytique
X_inputs = {
    'features': X_features,
    'mprime_g': data['mprime_g'].values.reshape(-1, 1),
    'mprime_a': data['mprime_a'].values.reshape(-1, 1),
    'GOD': data['GOD'].values.reshape(-1, 1),
    'AODS': data['AODS'].values.reshape(-1, 1),
    'Tg_abs': data['Tg_abs'].values.reshape(-1, 1),
    'Ta_abs': data['Ta_abs'].values.reshape(-1, 1),
}

# y = valeur cible (E_BOA / E_TOA) → entre 0 et 1
y = data['BOA_fraction'].values


In [53]:
# 1. Early stopping
early_stop = EarlyStopping(
    monitor='val_mae',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

# 2. Reduce learning rate on plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_mae',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

# 3. Model checkpoint (save best model)
checkpoint_cb = ModelCheckpoint(
    filepath='deep_model_1.keras',
    monitor='val_mae',
    save_best_only=True,
    verbose=1
)

# 4. CSV logger (log training history)
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
csv_logger = CSVLogger(f'training_log_{timestamp}.csv')

In [54]:
model.fit(
    X_inputs,
    y,
    batch_size=64,
    epochs=100,
    validation_split=0.2,
    callbacks=[early_stop, reduce_lr, checkpoint_cb, csv_logger]
)

Epoch 1/100
4524/4532 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5624 - mae: 0.5624
Epoch 1: val_mae improved from inf to 0.55872, saving model to deep_model_1.keras
4532/4532 ━━━━━━━━━━━━━━━━━━━━ 27s 6ms/step - loss: 0.5624 - mae: 0.5624 - val_loss: 0.5587 - val_mae: 0.5587 - learning_rate: 3.4000e-04
Epoch 2/100
4523/4532 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5572 - mae: 0.5572
Epoch 2: val_mae improved from 0.55872 to 0.55869, saving model to deep_model_1.keras
4532/4532 ━━━━━━━━━━━━━━━━━━━━ 26s 6ms/step - loss: 0.5572 - mae: 0.5572 - val_loss: 0.5587 - val_mae: 0.5587 - learning_rate: 3.4000e-04
Epoch 3/100
4521/4532 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5579 - mae: 0.5579
Epoch 3: val_mae improved from 0.55869 to 0.55869, saving model to deep_model_1.keras
4532/4532 ━━━━━━━━━━━━━━━━━━━━ 26s 6ms/step - loss: 0.5579 - mae: 0.5579 - val_loss: 0.5587 - val_mae: 0.5587 - learning_rate: 3.4000e-04
Epoch 4/100
4525/4532 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5572 - mae: 0.5

In [20]:
import tensorflow as tf
import json
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, Lambda

# Chargement des paramètres optimaux
with open("best_params.json", "r") as f:
    params = json.load(f)

# Fonction d'activation personnalisée qui ne reçoit les 6 paramètres qu'en sortie
def transmission_activation(inputs):
    alpha, m_g, m_a, dtau_gas_scat, dtau_aero_scat, T_gas_abs, T_aero_abs = inputs  # Décomposer la liste d'inputs
    
    # Dénominateur
    denom = 1 + alpha * dtau_gas_scat * m_g + (alpha / 3.0) * dtau_aero_scat * m_a
    
    # Absorption
    T_abs = tf.pow(T_gas_abs, m_g) * tf.pow(T_aero_abs, m_a)
    
    # Transmission finale
    return T_abs / denom

# Définition des entrées complètes
input_features = Input(shape=(X_train.shape[1],))  # Inclut toutes les colonnes pour l'entraînement

# Couches cachées
hidden = Dense(params["units1"], activation=params["activation"])(input_features)
hidden = Dropout(params["dropout1"])(hidden)

hidden = Dense(params["units2"], activation=params["activation"])(hidden)
hidden = Dropout(params["dropout2"])(hidden)

if params["n_layers"] >= 3:
    hidden = Dense(params["units3"], activation=params["activation"])(hidden)
    hidden = Dropout(params["dropout3"])(hidden)

if params["n_layers"] == 4:
    hidden = Dense(params["units4"], activation=params["activation"])(hidden)
    hidden = Dropout(params["dropout4"])(hidden)

# Transmission directe de la sortie avant activation
alpha = Lambda(lambda x: x)(hidden)  # Passage direct sans transformation

# Entrées des 6 paramètres après entraînement
input_m_g = Input(shape=(1,))
input_m_a = Input(shape=(1,))
input_dtau_gas_scat = Input(shape=(1,))
input_dtau_aero_scat = Input(shape=(1,))
input_T_gas_abs = Input(shape=(1,))
input_T_aero_abs = Input(shape=(1,))

# Application de la fonction analytique sur la sortie après entraînement
# Application de la fonction analytique sur la sortie
output = Lambda(lambda inputs: transmission_activation(inputs), output_shape=(None, 1))(
    [alpha, input_m_g, input_m_a, input_dtau_gas_scat, input_dtau_aero_scat, input_T_gas_abs, input_T_aero_abs]
)

# Définition du modèle final
model = Model(inputs=[input_features, input_m_g, input_m_a, input_dtau_gas_scat, input_dtau_aero_scat, input_T_gas_abs, input_T_aero_abs], outputs=output)

# Compilation du modèle
optimizer = tf.keras.optimizers.Adam(learning_rate=params["lr"])
model.compile(optimizer=optimizer, loss='mae', metrics=['mae'])

# Affichage du résumé du modèle
model.summary()


Model: "functional_22"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 232)       │      4,872 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 232)       │          0 │ dense_15[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 249)       │     58,017 │ dropout_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_13          │ (None, 249)       │          0 │ dense_16[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 174)       │     43,500 │ dropout_13[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 174)       │          0 │ dense_17[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 174)       │          0 │ dropout_14[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_8       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_9       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, None, 1)   │          0 │ lambda[0][0],     │
│                     │                   │            │ input_layer_4[0]… │
│                     │                   │            │ input_layer_5[0]… │
│                     │                   │            │ input_layer_6[0]… │
│                     │                   │            │ input_layer_7[0]… │
│                     │                   │            │ input_layer_8[0]… │
│                     │                   │            │ input_layer_9[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 106,389 (415.58 KB)

 Trainable params: 106,389 (415.58 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
import tensorflow as tf
import json
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, Lambda

# Chargement des meilleurs hyperparamètres
with open("best_params.json", "r") as f:
    params = json.load(f)

# ----------------------------
# Fonction de transmission personnalisée
# ----------------------------
def transmission_activation(inputs):
    alpha, m_g, m_a, dtau_gas_scat, dtau_aero_scat, T_gas_abs, T_aero_abs = inputs
    
    denom = 1 + alpha * dtau_gas_scat * m_g + (alpha / 3.0) * dtau_aero_scat * m_a
    T_abs = tf.pow(T_gas_abs, m_g) * tf.pow(T_aero_abs, m_a)
    
    return T_abs / denom

# ----------------------------
# Définition des entrées
# ----------------------------
input_features = Input(shape=(X_train.shape[1],), name="features")

input_m_g = Input(shape=(1,), name="m_g")
input_m_a = Input(shape=(1,), name="m_a")
input_dtau_gas_scat = Input(shape=(1,), name="dtau_gas_scat")
input_dtau_aero_scat = Input(shape=(1,), name="dtau_aero_scat")
input_T_gas_abs = Input(shape=(1,), name="T_gas_abs")
input_T_aero_abs = Input(shape=(1,), name="T_aero_abs")

# ----------------------------
# Réseau de neurones MLP
# ----------------------------
x = Dense(params["units1"], activation=params["activation"])(input_features)
x = Dropout(params["dropout1"])(x)

x = Dense(params["units2"], activation=params["activation"])(x)
x = Dropout(params["dropout2"])(x)

if params["n_layers"] >= 3:
    x = Dense(params["units3"], activation=params["activation"])(x)
    x = Dropout(params["dropout3"])(x)

if params["n_layers"] == 4:
    x = Dense(params["units4"], activation=params["activation"])(x)
    x = Dropout(params["dropout4"])(x)

# ----------------------------
# Dernier neurone = alpha
# ----------------------------
alpha = Dense(1, name="alpha")(x)

# ----------------------------
# Couche de sortie : transmission personnalisée
# ----------------------------
output = Lambda(lambda inputs: transmission_activation(inputs), output_shape=(1,))(
    [alpha, input_m_g, input_m_a, input_dtau_gas_scat, input_dtau_aero_scat, input_T_gas_abs, input_T_aero_abs]
)

# ----------------------------
# Modèle final
# ----------------------------
model = Model(
    inputs=[
        input_features,
        input_m_g,
        input_m_a,
        input_dtau_gas_scat,
        input_dtau_aero_scat,
        input_T_gas_abs,
        input_T_aero_abs
    ],
    outputs=output
)

# Compilation
optimizer = tf.keras.optimizers.Adam(learning_rate=params["lr"])
model.compile(optimizer=optimizer, loss='mae', metrics=['mae'])

# Résumé
model.summary()


Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ features            │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 232)       │      4,872 │ features[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 232)       │          0 │ dense_18[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_19 (Dense)    │ (None, 249)       │     58,017 │ dropout_15[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_16          │ (None, 249)       │          0 │ dense_19[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_20 (Dense)    │ (None, 174)       │     43,500 │ dropout_16[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_17          │ (None, 174)       │          0 │ dense_20[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ alpha (Dense)       │ (None, 1)         │        175 │ dropout_17[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ m_g (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ m_a (InputLayer)    │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dtau_gas_scat       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dtau_aero_scat      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ T_gas_abs           │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ T_aero_abs          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 1)         │          0 │ alpha[0][0],      │
│                     │                   │            │ m_g[0][0],        │
│                     │                   │            │ m_a[0][0],        │
│                     │                   │            │ dtau_gas_scat[0]… │
│                     │                   │            │ dtau_aero_scat[0… │
│                     │                   │            │ T_gas_abs[0][0],  │
│                     │                   │            │ T_aero_abs[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 106,564 (416.27 KB)

 Trainable params: 106,564 (416.27 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
data
X_train=data.drop(columns=['BOA_fraction'])
y_train=data['BOA_fraction']
y_train.shape

(362503,)

In [23]:
X_train_m_g = X_train["mprime_g"].values.reshape(-1, 1)
X_train_m_a = X_train["mprime_a"].values.reshape(-1, 1)
X_train_dtau_gas_scat = X_train["GOD"].values.reshape(-1, 1)
X_train_dtau_aero_scat = X_train["AODS"].values.reshape(-1, 1)
X_train_T_gas_abs = X_train["Tg_abs"].values.reshape(-1, 1)
X_train_T_aero_abs = X_train["Ta_abs"].values.reshape(-1, 1)


In [24]:
history = model.fit(
    [X_train, X_train_m_g, X_train_m_a, X_train_dtau_gas_scat, X_train_dtau_aero_scat, X_train_T_gas_abs, X_train_T_aero_abs], 
    y_train,
    batch_size=32,
    epochs=100
)


Epoch 1/100
11329/11329 ━━━━━━━━━━━━━━━━━━━━ 49s 4ms/step - loss: 0.5633 - mae: 0.5633
Epoch 2/100
11329/11329 ━━━━━━━━━━━━━━━━━━━━ 47s 4ms/step - loss: 0.5579 - mae: 0.5579
Epoch 3/100
11329/11329 ━━━━━━━━━━━━━━━━━━━━ 48s 4ms/step - loss: 0.5578 - mae: 0.5578
Epoch 4/100
11329/11329 ━━━━━━━━━━━━━━━━━━━━ 47s 4ms/step - loss: 0.5583 - mae: 0.5583
Epoch 5/100
 4660/11329 ━━━━━━━━━━━━━━━━━━━━ 29s 4ms/step - loss: 0.5584 - mae: 0.5584

KeyboardInterrupt: 